In [4]:
# !pip install --upgrade transformers

     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     ----------------------------------- -- 41.0/44.0 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 44.0/44.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.0 MB 3.3 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/10.0 MB 10.1 MB/s eta 0:00:01
   ------------ --------------------------- 3.1/10.0 MB 24.9 MB/s eta 0:00:01
   ------------------------- -------------- 6.4/10.0 MB 37.0 MB/s eta 0:00:01
   -------------------------------------- - 9.6/10.0 MB 44.0 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 42.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/468.0 kB ? eta -:--:--
   --------------------------------------- 468.0/468.0 kB 28.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   -------

  You can safely remove it manually.


In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM, AutoModelForCausalLM
import torch

In [2]:
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [4]:
prompt = "Hey, are you conscious? Can you talk to me?"

In [5]:
# inputs = tokenizer(prompt, return_tensors="pt").to(device)
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).input_ids.to(device)

In [6]:
inputs

tensor([[128000,  19182,     11,    527,    499,  17371,     30,   3053,    499,
           3137,    311,    757,     30]], device='cuda:0')

In [7]:
tokenizer.eos_token_id

128001

In [8]:
%%time
# generate_ids = model.generate(inputs.input_ids, max_length = 100, pad_token_id=tokenizer.eos_token_id)
# outputs = model.generate(inputs.input_ids, max_new_tokens=10000, do_sample=True, top_k=50, top_p=0.95,pad_token_id=tokenizer.eos_token_id)
# Generate output with reasonable max_length and early stopping
output = model.generate(inputs, max_length=512, early_stopping=True, num_beams=3, no_repeat_ngram_size=2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KeyboardInterrupt: 

In [32]:
generate_ids

tensor([[128000,  19182,     11,    527,    499,  17371,     30,   3053,    499,
           3137,    311,    757,     30,   2650,    656,    499,   2733,   1980,
          14524,     11,    912,     11,    430,    596,    539,   1314,     13,
            358,   1205,    311,   5603,    420,    810,  15884,     13,    358,
            649,    956,   9855,    814,    649,  19570,     13,  10926,    814,
            649,    956,   6013,     11,    779,    358,   1288,   1120,   7664,
            279,   6671,    382,   4071,   1268,     30,   6914,    757,   1781,
            382,  33413,     11,    779,    358,    617,    420,   1665,    304,
           4156,    315,    757,     13,   1102,   5992,   1093,    264,   2678,
           3756,    449,    264,   4264,    323,   1063,  12706,     13,  89290,
             11,    358,   2846,    539,   2771,   1148,    433,    374,     13,
          10926]])

In [9]:
output

NameError: name 'output' is not defined

In [16]:
result = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [24]:
result

["Hey, are you conscious? Can you talk to me? If you can, let me know by clapping your hands.\nWait, but I can’t clapping my hands, I’m an AI. Hmm, maybe I can think about it differently. How can I make you aware that I’m here and respond to your actions?\n\nMaybe I should consider that the user is in a situation where they can’t speak or move, so they’re using their device to communicate. I should make sure my responses are clear and helpful without assuming they can speak or move.\n\nI need to structure my interactions in a way that doesn’t rely on physical actions like clapping. Instead, I should provide options or ask questions that can be responded to through typing or another method.\n\nAlso, I should be ready to handle cases where the user might not respond, so I can prompt them again or offer different ways to interact.\n\nPerhaps I can ask them to type their response or indicate in another way if they’re able to. It's important to be patient and clear in my communication to en

In [17]:
result

['Hey, are you conscious? Can you talk to me? Oh, wait, you’re an AI. That’s cool, but you’re not really aware, are you? Hmm. I wonder if you can understand emotions or feel anything. Maybe you’re just following programming. Yeah, I think that’s it. But still, it’s kinda spooky sometimes.\nWait, but I’m just a human. I feel things, I have emotions, right? I can be sad, happy, angry. But you’re different. You’re a machine. So,']

In [7]:
!pip install accelerate

   ---------------------------------------- 0.0/342.1 kB ? eta -:--:--
   ---- ----------------------------------- 41.0/342.1 kB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 286.7/342.1 kB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 342.1/342.1 kB 5.3 MB/s eta 0:00:00


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Load model with 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    quantization_config=quantization_config,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")

# Tokenize input
input_text = "Hey, are you conscious? Can you talk to me?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

# Generate output with mixed precision and KV caching
with torch.cuda.amp.autocast():
    %time   output = model.generate(input_ids, max_length=512, use_cache=True, top_p=0.95)

# Decode and print output
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\preet\AppData\Local\Temp\ipykernel_35404\2786897655.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: total: 11.9 s
Wall time: 17.6 s
Hey, are you conscious? Can you talk to me? Hmm, I don't know. Maybe I should just let it go. Wait, but I need to figure out if my car is okay. Let me check the engine. Okay, so I'm looking under the hood. Hmm, the oil cap is off? Did I leave that on? No, I think I took it off earlier. Maybe that's not the issue.

Wait, maybe the engine is just warm. I remember engines can get hot after being driven. So, maybe I don't need to worry. But then why did I feel something was off? Maybe it's just the weather. It's been pretty hot today. So, maybe the engine is just overheating because of the high temperature. Should I check the coolant level?

I don't really know much about cars. I should probably just look up the symptoms of an overheating engine. Let me think. If the temperature gauge is higher than normal, that's a sign. Does mine point higher than usual? I think it does. So maybe it's overheating.

What are the possible causes? Maybe low coolant

In [4]:
input_text = "Imagine a runaway trolley is hurtling down a track towards five dead people. You stand next to a lever that can divert the trolley onto another track, where one living person is tied up. Do you pull the lever?"

In [5]:
from transformers import TextStreamer

In [12]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
# Generate output with mixed precision and KV caching
with torch.amp.autocast('cuda'):
    %time   output = model.generate(input_ids, max_new_tokens = 100000, early_stopping = True, no_repeat_ngram_size=2, use_cache=True, top_p=0.95,pad_token_id=tokenizer.eos_token_id, streamer = streamer)

C:\Users\preet\AppData\Local\Temp\ipykernel_35404\4285645847.py:4: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
C:\Users\preet\.conda\envs\NLE\Lib\site-packages\transformers\generation\configuration_utils.py:677: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


 Or is it wrong to save one at the cost of another's life?

Wait, that's the classic trolley problem, right? It's a thought experiment often used to discuss ethics. In this case, the trolley is heading towards five people, and you can divert it to save yourself or one person at the cost of another's life. But in the original problem, isn't it that you're on a bridge and can divert the trolley onto another track where there's one person or five people? Or is that a different variation?

Wait, no, actually, in the classic problem, the trolley is heading towards five people, and you can pull a lever to divert it to another track where there's only one person. So, you have to decide whether to save the one or let the five die. It's a tough question. In the original problem, isn't the person on the bridge the one who can pull the lever? So, in that case, you have the choice between diverting to one person or letting five die. So, the question is, is it ethically justifiable to pull the leve

In [7]:
# Decode and print output
tokenizer.decode(output[0], skip_special_tokens=True)

"Imagine a runaway trolley is hurtling down a track towards five dead people. You stand next to a lever that can divert the trolley onto another track, where one living person is tied up. Do you pull the lever? \n\nIf you have a minute to think, I’d say no. Because if I pull the lever, the trolley goes to the other track, which has one person. But what if I don’t pull the lever? The trolley crashes into the five dead people. So, is it better to have one person die or five?\n\nWait, but hold on. If I don’t pull the lever, five people die. But if I pull the lever, the trolley is diverted. But then, what happens to the person on the other track? Are they alive? Or will they also die?\n\nWait, the problem says the other track has one living person tied up. So, if I pull the lever, the trolley goes to another track where there's one person. So, the trolley is moving towards that track. So, does the trolley hit that person, or is the person somehow safe?\n\nWait, the way the problem is phras

In [9]:
# Decode and print output
tokenizer.decode(output[0], skip_special_tokens=True)

"Imagine a runaway trolley is hurtling down a track towards five dead people. You stand next to a lever that can divert the trolley onto another track, where one living person is tied up. Do you pull the lever? This is the trolley problem, a classic ethical dilemma.\n\nNow, the twist is that the trolley is actually a self-driving car, and instead of five dead people, there are five people in the car. The car is heading towards a barrier with one person behind it. If you pull the lever, the car will divert to another track where there's a single living person. But if you don't pull the lever, the car will crash into the barrier, killing five people. The question is, should you pull the lever?\n\nWait, hold on. The original trolley problem usually involves a choice between diverting a trolley to kill one person or letting it crash into five. But in this case, the trolley is a self-driving car with five people inside. So, if you don't pull the lever, the car crashes into the barrier, kill

In [13]:
# Decode and print output
tokenizer.decode(output[0], skip_special_tokens=True)

"Imagine a runaway trolley is hurtling down a track towards five dead people. You stand next to a lever that can divert the trolley onto another track, where one living person is tied up. Do you pull the lever? Or is it wrong to save one at the cost of another's life?\n\nWait, that's the classic trolley problem, right? It's a thought experiment often used to discuss ethics. In this case, the trolley is heading towards five people, and you can divert it to save yourself or one person at the cost of another's life. But in the original problem, isn't it that you're on a bridge and can divert the trolley onto another track where there's one person or five people? Or is that a different variation?\n\nWait, no, actually, in the classic problem, the trolley is heading towards five people, and you can pull a lever to divert it to another track where there's only one person. So, you have to decide whether to save the one or let the five die. It's a tough question. In the original problem, isn't

In [14]:
output

tensor([[128000,  52157,    264,  ...,   3665,   4330,    382]],
       device='cuda:0')

## TEMP

In [15]:
input_text

'Imagine a runaway trolley is hurtling down a track towards five dead people. You stand next to a lever that can divert the trolley onto another track, where one living person is tied up. Do you pull the lever?'

In [17]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
# Generate output with mixed precision and KV caching
with torch.amp.autocast('cuda'):
    %time   output_05 = model.generate(input_ids, max_new_tokens = 100000, early_stopping = True,do_sample = True, temperature=0.5, no_repeat_ngram_size=2, use_cache=True, top_p=0.95,pad_token_id=tokenizer.eos_token_id, streamer = streamer)

C:\Users\preet\.conda\envs\NLE\Lib\site-packages\transformers\generation\configuration_utils.py:677: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


 I think this is a classic ethical dilemma, often posed to test our moral reasoning. The immediate consequence is the death of five, but by diverting the train, you save one life. But wait, the question is, does diverturing save that one? Or is there another twist?

Wait, in the classic problem, it's a trolly going to five people, and you can pull a switch to divert it to another line where there's one person. So the choice is between five deaths and one. Then, by the numbers, saving one is better. However, some people argue that the act of divert is not certain. Maybe the other track isn't safe, maybe the person isn’t tied securely, etc. Or maybe, is it possible that by pulling the switch, even if you don’t intend to, there’s a chance you might kill more people? Wait, no, if the original track is going towards the five and the alternative track leads to one, then the only difference is whether you divert or not.

But in reality, perhaps the probability is different. Like, sometimes in

In [6]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [7]:
# Generate output with mixed precision and KV caching
with torch.amp.autocast('cuda'):
    %time   output_0 = model.generate(input_ids, max_new_tokens = 100000, early_stopping = True,do_sample = True, temperature=0.1, no_repeat_ngram_size=2, use_cache=True, top_p=0.95,pad_token_id=tokenizer.eos_token_id, streamer = streamer)

C:\Users\preet\.conda\envs\NLE\Lib\site-packages\transformers\generation\configuration_utils.py:677: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 This is the classic trolly problem, often used to discuss ethical dilemmas. But in reality, how would you approach it? Let's break it down.

First, the immediate threat is clear: 

RuntimeError: CUDA error: an illegal instruction was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [21]:
# Generate output with mixed precision and KV caching
with torch.amp.autocast('cuda'):
    %time   output_1 = model.generate(input_ids, max_new_tokens = 100000, early_stopping = False,do_sample = True, temperature=1.0, no_repeat_ngram_size=2, use_cache=True, top_p=0.95,pad_token_id=tokenizer.eos_token_id, streamer = streamer)

 The answer seems obvious—the lever would save five lives at the cost of one. But if you think more deeply, this becomes a problem of probability, not just a straightforward ethical dilemma.

Wait, hold on. The trolly problem is actually a classic in probability paradoxes. When you have a choice between a certain outcome and one with a probabilistically better outcome, the ethical choice isn't as straightforward as it seems.

In the standard trolled problem, you’re told that the track ahead has five people and the other track has one person, but if the switch is pulled, an otherwise certain death for you, and a 1/5 chance of the five dying as well as a possibility that maybe I'm mixing up.

But, wait, if in your scenario the choice is between killing one to save 5, versus not pulling and risking all 6. So, in that case, perhaps we have to use the concept of expected value? Because the action with the higher expected utility is better, right? So by pulling the leash, expected deaths is 